In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# Descargar e instalar Spark
!wget -q https://dlcdn.apache.org/spark/spark-3.5.2/spark-3.5.2-bin-hadoop3.tgz
!tar xf spark-3.5.2-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

In [ ]:
# Crear la sesión de Spark
spark = SparkSession.builder.master("local").\
    appName("ITESO-SVM").\
    getOrCreate()

# Configurar el contexto de Spark
sc = spark.sparkContext
sc.setLogLevel("ERROR")

# Cargar el conjunto de datos de Iris
data_path = "/content/drive/MyDrive/Colab Notebooks/datasets/iris.csv"
iris_df = spark.read.csv(data_path, header=True, inferSchema=True)
iris_df.show(5)

In [ ]:
indexer = StringIndexer(inputCol="Species", outputCol="label")
iris_df = indexer.fit(iris_df).transform(iris_df)

In [ ]:
feature_columns = ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
iris_df = assembler.transform(iris_df).select("label", "features")
iris_df.show(5)

In [ ]:
train_data, test_data = iris_df.randomSplit([0.8, 0.2], seed=1523)

In [ ]:
lsvc = LinearSVC(maxIter=10, regParam=0.01)  # Cambiamos de DecisionTreeClassifier a LinearSVC
ovr = OneVsRest(classifier=lsvc)  # Añadimos OneVsRest para la clasificación multiclase


In [ ]:
ovr_model = ovr.fit(train_data)

In [ ]:
predictions = ovr_model.transform(test_data)
predictions.select("label", "prediction").show(5)

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")  # Cambiamos a evaluación F1
f1_score = evaluator.evaluate(predictions)
print(f"F1 Score: {f1_score}")